# Transfer Learning
* Pretrained ResNet50
* Custom dataset
* Add fully connected layer

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.datasets as Dataset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# 1. Hyperparameters

In [39]:
batch_size = 6
learning_rate = 0.001
epoch = 5
num_category = 2

# 2. Data
## 1) Load images from folder 

In [3]:
img_dir = "./images"
# 여기서 중요한 것은 라벨링은 images 폴더 아래
# 폴더로 라벨링이 된다는 것이다.
img_data = Dataset.ImageFolder(img_dir, transforms.Compose([
            transforms.RandomResizedCrop (224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            ]))

print(img_data.classes)
print(img_data.class_to_idx)
print(img_data.imgs)


['dogs', 'onepun']
{'dogs': 0, 'onepun': 1}
[('./images\\dogs\\3b1a8d3fca2adb68a7a908b73005764a.jpg', 0), ('./images\\dogs\\dog-650299.jpg', 0), ('./images\\dogs\\retriever.jpg', 0), ('./images\\onepun\\1454.jpg', 1), ('./images\\onepun\\5320.jpg', 1), ('./images\\onepun\\5720.jpg', 1)]


## 2) Set data loader

In [4]:
img_batch = DataLoader(img_data, batch_size = batch_size,
                         shuffle=True, num_workers=2)

for img,label in img_batch:
    print(img.size())
    print(label)

torch.Size([6, 3, 224, 224])
tensor([1, 0, 0, 0, 1, 1])


# 3. ResNet Model & Optimizer

In [6]:
resnet = models.resnet50(pretrained=True)

for name, module in resnet.named_children():
    print(name)

conv1
bn1
relu
maxpool
layer1
layer2
layer3
layer4
avgpool
fc


In [7]:
print(resnet.children())

<generator object Module.children at 0x000001B500048A98>


# 2) Fully Connected Model

In [8]:
class Resnet(nn.Module):
    def __init__(self):
        super(Resnet, self).__init__()
                      # 여기 부분을 기억하자. 미리 학습된 정보를 때려넣는다. 
        self.layer0 = nn.Sequential(*list(resnet.children()) [0:-1] )
        self.layer1 = nn.Sequential(
            nn.Linear(2048, 500),
            nn.BatchNorm1d(500),
            nn.ReLU(),
            nn.Linear(500, num_category),
            nn.ReLU()
        )
        
    def forward(self,x):
        out = self.layer0(x)
        out = out.view(batch_size, -1)
        out = self.layer1(out)
        
        return out

In [9]:
model = Resnet().cuda()

# 이부분은 프리_Trained 라서 그렇다.
for params in model.layer0.parameters():
    params.requires_grad = False # layer0의 파라미터는 학습을 시키지 않는다. 

for params in model.layer1.parameters():
    params.requires_grad = True

In [10]:
for name in model.children():
    print(name)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1

# 4) Loss & Optimizer

In [20]:
# define loss func & optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.layer1.parameters(), lr=learning_rate)
print(img_batch.dataset.imgs)

[('./images\\dogs\\3b1a8d3fca2adb68a7a908b73005764a.jpg', 0), ('./images\\dogs\\dog-650299.jpg', 0), ('./images\\dogs\\retriever.jpg', 0), ('./images\\onepun\\1454.jpg', 1), ('./images\\onepun\\5320.jpg', 1), ('./images\\onepun\\5720.jpg', 1)]


# 4. Train

In [42]:
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# labels = item['label'].to(device) # GPU로 돌리자 @@

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# torch.max가 어떤 역할을 할까?
# 나오는 출력의 첫 번째 tensor => 값 !
# 나오는 출력의 두 번째 tensor => 인덱스

for i in range(epoch):
    
    for i_batch, (img, label) in enumerate(img_batch):
        img = img.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()
        output = model(img)
        loss = loss_func(output, label)
        loss.backward()
        optimizer.step() 
        
    print('Epoch [{}/{}], Loss: {:.4f}'.format(i+ 1, epoch, loss.item()))
    
        
        

Epoch [1/5], Loss: 0.0000
Epoch [2/5], Loss: 0.0000
Epoch [3/5], Loss: 0.0000
Epoch [4/5], Loss: 0.0000
Epoch [5/5], Loss: 0.0000


# 5. Test

In [43]:
model.eval()
correct = 0
total = 0

for img,label in img_batch:
    img = img.to(device)
    label = label.to(device)
    
    output = model(img)
    _, pred = torch.max(output.data, 1)
    
    total += label.size(0)
    correct += (pred == label.data).sum()
    
print("Accuracy: {}".format(correct/total))

Accuracy: 1
